In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')


อ่าน application data

In [2]:
# Testing data features
app_test = pd.read_csv('../input/application_test.csv')
print('Testing data shape: ', app_test.shape)
app_test.head()

Testing data shape:  (48744, 121)


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Training data
app_train = pd.read_csv('../input/application_train.csv')
print('Training data shape: ', app_train.shape)
app_train.head()

Training data shape:  (307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


ดูว่าแต่ละ columns มีประเภทอะไรบ้าง

In [4]:
# Number of each type of column
app_train.dtypes.value_counts()

float64    65
int64      41
object     16
dtype: int64

In [5]:
# Number of unique classes in each object column
app_train.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

NAME_CONTRACT_TYPE             2
CODE_GENDER                    3
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
NAME_TYPE_SUITE                7
NAME_INCOME_TYPE               8
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
OCCUPATION_TYPE               18
WEEKDAY_APPR_PROCESS_START     7
ORGANIZATION_TYPE             58
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
WALLSMATERIAL_MODE             7
EMERGENCYSTATE_MODE            2
dtype: int64

#### Manual target encoder

In [6]:
def TargetEncoder(app_train,app_test) :
    for col in app_train :
        if app_train[col].dtype == 'object':
            cumsum = app_train.groupby(col)['TARGET'].sum()
            cumcnt = app_train.groupby(col)['TARGET'].count()
            for x in list(cumsum.index) :
                c = cumsum.loc[x]/cumcnt.loc[x]
                app_train.loc[app_train[col] == x, col] = c
                app_test.loc[app_test[col] == x, col] = c
            app_train[col] = app_train[col].astype('float64')
            app_test[col] = app_test[col].astype('float64')

Encode app_train กับ app_test 

In [7]:
TargetEncoder(app_train,app_test)

In [8]:
app_train.head(5)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,0.083459,0.101419,0.085002,0.079616,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,0.083459,0.069993,0.085002,0.083249,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,0.054783,0.101419,0.072437,0.079616,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,0.083459,0.069993,0.085002,0.079616,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,0.083459,0.101419,0.085002,0.079616,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
app_train.dtypes.value_counts()

float64    81
int64      41
dtype: int64

In [10]:
app_test.dtypes.value_counts()

float64    81
int64      40
dtype: int64

### Bureau table

In [11]:
bureau = pd.read_csv('../input/bureau.csv')
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


ดูว่ามีการ loan ก่อนหน้านั้นหรือไม่

In [12]:
previous_loan_counts = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU' : 'previous_loan_counts'})

In [13]:
previous_loan_counts.head()

,SK_ID_CURR,previous_loan_counts
0,100001,7
1,100002,8
2,100003,4
3,100004,2
4,100005,3


In [14]:
# Join to the training dataframe
app_train = app_train.merge(previous_loan_counts, on = 'SK_ID_CURR', how = 'left')

# Join to the testing dataframe
app_test = app_test.merge(previous_loan_counts, on = 'SK_ID_CURR', how = 'left')

# Fill the missing values with 0 
app_train['previous_loan_counts'] = app_train['previous_loan_counts'].fillna(0)
app_test['previous_loan_counts'] = app_test['previous_loan_counts'].fillna(0)
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,previous_loan_counts
0,100002,1,0.083459,0.101419,0.085002,0.079616,0,202500.0,406597.5,24700.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,8.0
1,100003,0,0.083459,0.069993,0.085002,0.083249,0,270000.0,1293502.5,35698.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,100004,0,0.054783,0.101419,0.072437,0.079616,0,67500.0,135000.0,6750.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,100006,0,0.083459,0.069993,0.085002,0.079616,0,135000.0,312682.5,29686.5,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,100007,0,0.083459,0.101419,0.085002,0.079616,0,121500.0,513000.0,21865.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [15]:
app_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,previous_loan_counts
0,100001,0.083459,0.069993,0.085002,0.079616,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
1,100005,0.083459,0.101419,0.085002,0.079616,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0,3.0
2,100013,0.083459,0.101419,0.072437,0.079616,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0,4.0
3,100028,0.083459,0.069993,0.085002,0.079616,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0,12.0
4,100038,0.083459,0.101419,0.072437,0.083249,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0


# คำนวณ columns ที่เกี่ยวกับตัวเลข
เอาตาราง bureau มา Group ตาม SK_ID_CURR

In [16]:
def agg_numeric(df, group_var, df_name) :
    for col in df:
        if col != group_var and 'SK_ID' in col : # เป็น SK_ID แต่ไม่ใช่่สิ่งที่ต้องการจะ group ก็ให้ลบทิ้ง col นั้นไปเลย
            df = df.drop(columns = col)
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number') # ทำตารางใหม่ เอาข้อมูลที่เป็น ตัวเลข เท่านั้น
    numeric_df[group_var] = group_ids # ใส่สิ่งที่จะ group ไปด้วย
    
    agg = numeric_df.groupby(group_var).agg(['count','mean','max','min','sum']).reset_index()
    
    columns = [group_var] # เปลี่ยนชื่อตาราง
    for var in agg.columns.levels[0]:
        if var != group_var:
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))
    agg.columns = columns
    
    # Remove the columns with all redundant values
    _, idx = np.unique(agg, axis = 1, return_index=True)
    agg = agg.iloc[:,idx]
    
    return agg

In [17]:
bureau_agg_new = agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), group_var='SK_ID_CURR', df_name='bureau')
bureau_agg_new.head()

,bureau_DAYS_CREDIT_sum,bureau_DAYS_ENDDATE_FACT_sum,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_ENDDATE_min,bureau_DAYS_ENDDATE_FACT_min,bureau_DAYS_ENDDATE_FACT_mean,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_UPDATE_sum,bureau_DAYS_ENDDATE_FACT_max,bureau_DAYS_CREDIT_UPDATE_min,...,bureau_AMT_CREDIT_SUM_min,SK_ID_CURR,bureau_AMT_CREDIT_SUM_mean,bureau_AMT_CREDIT_SUM_DEBT_max,bureau_AMT_CREDIT_SUM_max,bureau_AMT_CREDIT_SUM_DEBT_sum,bureau_AMT_CREDIT_SUM_sum,bureau_AMT_CREDIT_MAX_OVERDUE_min,bureau_AMT_CREDIT_MAX_OVERDUE_mean,bureau_AMT_CREDIT_MAX_OVERDUE_max
0,-5145,-3302.0,-1572,-1329.0,-1328.0,-825.500000,-735.000000,-652,-544.0,-155,...,85500.0,100001,207623.571429,373239.0,378000.0,596686.5,1453365.000,NaN,NaN,NaN
1,-6992,-4185.0,-1437,-1072.0,-1185.0,-697.500000,-874.000000,-3999,-36.0,-1185,...,0.0,100002,108131.945625,245781.0,450000.0,245781.0,865055.565,0.0,1681.029,5043.645
2,-5603,-3292.0,-2586,-2434.0,-2131.0,-1097.333333,-1400.750000,-3264,-540.0,-2131,...,22248.0,100003,254350.125000,0.0,810000.0,0.0,1017400.500,0.0,0.000,0.000
3,-1734,-1065.0,-1326,-595.0,-683.0,-532.500000,-867.000000,-1064,-382.0,-682,...,94500.0,100004,94518.900000,0.0,94537.8,0.0,189037.800,0.0,0.000,0.000
4,-572,-123.0,-373,-128.0,-123.0,-123.000000,-190.666667,-163,-123.0,-121,...,29826.0,100005,219042.000000,543087.0,568800.0,568408.5,657126.000,0.0,0.000,0.000


In [18]:
app_train = app_train.merge(bureau_agg_new, on = 'SK_ID_CURR', how='left')
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,bureau_AMT_CREDIT_SUM_DEBT_mean,bureau_AMT_CREDIT_SUM_min,bureau_AMT_CREDIT_SUM_mean,bureau_AMT_CREDIT_SUM_DEBT_max,bureau_AMT_CREDIT_SUM_max,bureau_AMT_CREDIT_SUM_DEBT_sum,bureau_AMT_CREDIT_SUM_sum,bureau_AMT_CREDIT_MAX_OVERDUE_min,bureau_AMT_CREDIT_MAX_OVERDUE_mean,bureau_AMT_CREDIT_MAX_OVERDUE_max
0,100002,1,0.083459,0.101419,0.085002,0.079616,0,202500.0,406597.5,24700.5,...,49156.2,0.0,108131.945625,245781.0,450000.0,245781.0,865055.565,0.0,1681.029,5043.645
1,100003,0,0.083459,0.069993,0.085002,0.083249,0,270000.0,1293502.5,35698.5,...,0.0,22248.0,254350.125000,0.0,810000.0,0.0,1017400.500,0.0,0.000,0.000
2,100004,0,0.054783,0.101419,0.072437,0.079616,0,67500.0,135000.0,6750.0,...,0.0,94500.0,94518.900000,0.0,94537.8,0.0,189037.800,0.0,0.000,0.000
3,100006,0,0.083459,0.069993,0.085002,0.079616,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,0.083459,0.101419,0.085002,0.079616,0,121500.0,513000.0,21865.5,...,0.0,146250.0,146250.000000,0.0,146250.0,0.0,146250.000,0.0,0.000,0.000


In [19]:
app_test = app_test.merge(bureau_agg_new, on = 'SK_ID_CURR', how='left')
app_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,bureau_AMT_CREDIT_SUM_DEBT_mean,bureau_AMT_CREDIT_SUM_min,bureau_AMT_CREDIT_SUM_mean,bureau_AMT_CREDIT_SUM_DEBT_max,bureau_AMT_CREDIT_SUM_max,bureau_AMT_CREDIT_SUM_DEBT_sum,bureau_AMT_CREDIT_SUM_sum,bureau_AMT_CREDIT_MAX_OVERDUE_min,bureau_AMT_CREDIT_MAX_OVERDUE_mean,bureau_AMT_CREDIT_MAX_OVERDUE_max
0,100001,0.083459,0.069993,0.085002,0.079616,0,135000.0,568800.0,20560.5,450000.0,...,85240.928571,85500.00,207623.571429,373239.0,378000.0,596686.5,1453365.00,NaN,NaN,NaN
1,100005,0.083459,0.101419,0.085002,0.079616,0,99000.0,222768.0,17370.0,180000.0,...,189469.500000,29826.00,219042.000000,543087.0,568800.0,568408.5,657126.00,0.0,0.0,0.0
2,100013,0.083459,0.101419,0.072437,0.079616,0,202500.0,663264.0,69777.0,630000.0,...,0.000000,26490.06,518070.015000,0.0,1262250.0,0.0,2072280.06,19305.0,19305.0,19305.0
3,100028,0.083459,0.069993,0.085002,0.079616,2,315000.0,1575000.0,49018.5,1575000.0,...,18630.450000,0.00,126739.590000,151015.5,393750.0,186304.5,1520875.08,0.0,0.0,0.0
4,100038,0.083459,0.101419,0.072437,0.083249,1,180000.0,625500.0,32067.0,625500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


ฟังก์ชันสำหรับการหา correlations ระหว่างข้อมูลในตารางกับ target

In [20]:
# Function to calculate correlations with the target for a dataframe
def target_corrs(df):

    # List of correlations
    corrs = []

    # Iterate through the columns 
    for col in df.columns:
        print(col)
        # Skip the target column
        if col != 'TARGET':
            # Calculate correlation with the target
            corr = df['TARGET'].corr(df[col])

            # Append the list as a tuple
            corrs.append((col, corr))
            
    # Sort by absolute magnitude of correlations
    corrs = sorted(corrs, key = lambda x: abs(x[1]), reverse = True)
    
    return corrs

# การจัดการสำหรับข้อมูลแบบ Categorical

จะใช้วิธี Target Encoder

function RenameColumn สำหรับการแปลงชื่อ column หลังจาก aggregate แล้ว

In [21]:
def RenameColumn(df, group_var, df_name):
    # Need to create new column names
    columns = []

    # Iterate through the variables names
    for var in df.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in ['count', 'count_norm']:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))

    #  Rename the columns
    df.columns = columns

In [22]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [23]:
bureau.shape

(1716428, 17)

In [24]:
#test_bureau.columns.to_series().groupby(test_bureau.dtypes).groups

In [25]:
app_train.dtypes.value_counts()

float64    138
int64       41
dtype: int64

In [26]:
train_cols = ['SK_ID_CURR','TARGET']
app_train_temp = app_train.copy()
app_train_temp = app_train_temp[train_cols]
app_train_temp.head()

,SK_ID_CURR,TARGET
0,100002,1
1,100003,0
2,100004,0
3,100006,0
4,100007,0


In [27]:
test_cols = ['SK_ID_CURR']
app_test_temp = app_test.copy()
app_test_temp = app_test_temp[test_cols]
app_test_temp.head()

,SK_ID_CURR
0,100001
1,100005
2,100013
3,100028
4,100038


In [28]:
app_train_temp.shape

(307511, 2)

เลือก column ที่เป็น object มาคิด targetEncoder

In [29]:
skIdCurr = bureau['SK_ID_CURR']
bureau_obj_cols = bureau.select_dtypes('object')
bureau_obj_cols['SK_ID_CURR'] = skIdCurr
app_train_temp_bureau_obj_cols = app_train_temp.merge(bureau_obj_cols, how="left")
app_train_temp_bureau_obj_cols.head()

,SK_ID_CURR,TARGET,CREDIT_ACTIVE,CREDIT_CURRENCY,CREDIT_TYPE
0,100002,1,Closed,currency 1,Credit card
1,100002,1,Closed,currency 1,Credit card
2,100002,1,Closed,currency 1,Consumer credit
3,100002,1,Closed,currency 1,Consumer credit
4,100002,1,Closed,currency 1,Consumer credit


In [30]:
app_test_temp_bureau_obj_cols = app_test_temp.merge(bureau_obj_cols, how="left")
app_test_temp_bureau_obj_cols.head()

,SK_ID_CURR,CREDIT_ACTIVE,CREDIT_CURRENCY,CREDIT_TYPE
0,100001,Closed,currency 1,Consumer credit
1,100001,Closed,currency 1,Consumer credit
2,100001,Closed,currency 1,Consumer credit
3,100001,Closed,currency 1,Consumer credit
4,100001,Active,currency 1,Consumer credit


In [31]:
print(app_train_temp_bureau_obj_cols.shape)
print(app_train.shape)

(1509345, 5)
(307511, 179)


In [32]:
TargetEncoder(app_train_temp_bureau_obj_cols, app_test_temp_bureau_obj_cols)
app_train_temp_bureau_obj_cols.head()

,SK_ID_CURR,TARGET,CREDIT_ACTIVE,CREDIT_CURRENCY,CREDIT_TYPE
0,100002,1,0.069311,0.078174,0.088009
1,100002,1,0.069311,0.078174,0.088009
2,100002,1,0.069311,0.078174,0.074837
3,100002,1,0.069311,0.078174,0.074837
4,100002,1,0.069311,0.078174,0.074837


จำแนกเป็น sum และ mean

In [33]:
app_train_bureau_group = app_train_temp_bureau_obj_cols.drop(columns="TARGET").groupby('SK_ID_CURR').agg(['sum', 'mean'])
app_test_bureau_group = app_test_temp_bureau_obj_cols.groupby('SK_ID_CURR').agg(['sum', 'mean'])

In [34]:
RenameColumn(app_train_bureau_group,'SK_ID_CURR','bureau')
RenameColumn(app_test_bureau_group,'SK_ID_CURR','bureau')

In [35]:
app_train_bureau_group.head()

,bureau_CREDIT_ACTIVE_count,bureau_CREDIT_ACTIVE_count_norm,bureau_CREDIT_CURRENCY_count,bureau_CREDIT_CURRENCY_count_norm,bureau_CREDIT_TYPE_count,bureau_CREDIT_TYPE_count_norm
SK_ID_CURR,,,,,,
100002,0.601636,0.075204,0.625392,0.078174,0.651381,0.081423
100003,0.300818,0.075204,0.312696,0.078174,0.325690,0.081423
100004,0.138622,0.069311,0.156348,0.078174,0.149673,0.074837
100006,0.000000,NaN,0.000000,NaN,0.000000,NaN
100007,0.069311,0.069311,0.078174,0.078174,0.074837,0.074837


In [36]:
app_test_bureau_group.head()

,bureau_CREDIT_ACTIVE_count,bureau_CREDIT_ACTIVE_count_norm,bureau_CREDIT_CURRENCY_count,bureau_CREDIT_CURRENCY_count_norm,bureau_CREDIT_TYPE_count,bureau_CREDIT_TYPE_count_norm
SK_ID_CURR,,,,,,
100001,0.555898,0.079414,0.547218,0.078174,0.523856,0.074837
100005,0.255080,0.085027,0.234522,0.078174,0.237682,0.079227
100013,0.277244,0.069311,0.312696,0.078174,0.261135,0.065284
100028,0.949601,0.079133,0.938087,0.078174,0.963899,0.080325
100038,0.000000,NaN,0.000000,NaN,0.000000,NaN


In [37]:
app_train_bureau_group = app_train_temp.merge(app_train_bureau_group, left_on = 'SK_ID_CURR', right_index = True, how="left")
app_test_bureau_group = app_test_temp.merge(app_test_bureau_group, left_on = 'SK_ID_CURR', right_index = True, how="left")

In [38]:
app_train_bureau_group.head()

,SK_ID_CURR,TARGET,bureau_CREDIT_ACTIVE_count,bureau_CREDIT_ACTIVE_count_norm,bureau_CREDIT_CURRENCY_count,bureau_CREDIT_CURRENCY_count_norm,bureau_CREDIT_TYPE_count,bureau_CREDIT_TYPE_count_norm
0,100002,1,0.601636,0.075204,0.625392,0.078174,0.651381,0.081423
1,100003,0,0.300818,0.075204,0.312696,0.078174,0.325690,0.081423
2,100004,0,0.138622,0.069311,0.156348,0.078174,0.149673,0.074837
3,100006,0,0.000000,NaN,0.000000,NaN,0.000000,NaN
4,100007,0,0.069311,0.069311,0.078174,0.078174,0.074837,0.074837


In [39]:
app_test_bureau_group.head()

,SK_ID_CURR,bureau_CREDIT_ACTIVE_count,bureau_CREDIT_ACTIVE_count_norm,bureau_CREDIT_CURRENCY_count,bureau_CREDIT_CURRENCY_count_norm,bureau_CREDIT_TYPE_count,bureau_CREDIT_TYPE_count_norm
0,100001,0.555898,0.079414,0.547218,0.078174,0.523856,0.074837
1,100005,0.255080,0.085027,0.234522,0.078174,0.237682,0.079227
2,100013,0.277244,0.069311,0.312696,0.078174,0.261135,0.065284
3,100028,0.949601,0.079133,0.938087,0.078174,0.963899,0.080325
4,100038,0.000000,NaN,0.000000,NaN,0.000000,NaN


function targetEncoder สำหรับตาราง bureau โดยเฉพาะ จากการทดลองก่อนหน้านี้

In [40]:
def count_categoricalForBureau(app_train, app_test, bureau, group_var, df_name) :
    train_cols = ['TARGET'] + [group_var]
    app_train_temp = app_train.copy()
    app_train_temp = app_train_temp[train_cols]
    test_cols = [group_var]
    app_test_temp = app_test.copy()
    app_test_temp = app_test_temp[test_cols]
    
    skIdCurr = bureau[group_var]
    bureau_obj_cols = bureau.select_dtypes('object')
    bureau_obj_cols[group_var] = skIdCurr
    app_train_temp_bureau_obj_cols = app_train_temp.merge(bureau_obj_cols, how="left")
    app_test_temp_bureau_obj_cols = app_test_temp.merge(bureau_obj_cols, how="left")
    TargetEncoder(app_train_temp_bureau_obj_cols, app_test_temp_bureau_obj_cols)
    
    app_train_bureau_group = app_train_temp_bureau_obj_cols.drop(columns="TARGET").groupby(group_var).agg(['sum', 'mean'])
    app_test_bureau_group = app_test_temp_bureau_obj_cols.groupby(group_var).agg(['sum', 'mean'])
    
    RenameColumn(app_train_bureau_group,group_var, df_name)
    RenameColumn(app_test_bureau_group,group_var, df_name)
    
    app_train_bureau_group = app_train_temp.merge(app_train_bureau_group, left_on = group_var, right_index = True, how="left")
    app_test_bureau_group = app_test_temp.merge(app_test_bureau_group, left_on = group_var, right_index = True, how="left")
    
    return app_train_bureau_group, app_test_bureau_group

In [41]:
app_train_bureau_group, app_test_bureau_group = count_categoricalForBureau(app_train, app_test, bureau, group_var='SK_ID_CURR', df_name='bureau')

In [42]:
app_train_bureau_group.head()

,TARGET,SK_ID_CURR,bureau_CREDIT_ACTIVE_count,bureau_CREDIT_ACTIVE_count_norm,bureau_CREDIT_CURRENCY_count,bureau_CREDIT_CURRENCY_count_norm,bureau_CREDIT_TYPE_count,bureau_CREDIT_TYPE_count_norm
0,1,100002,0.601636,0.075204,0.625392,0.078174,0.651381,0.081423
1,0,100003,0.300818,0.075204,0.312696,0.078174,0.325690,0.081423
2,0,100004,0.138622,0.069311,0.156348,0.078174,0.149673,0.074837
3,0,100006,0.000000,NaN,0.000000,NaN,0.000000,NaN
4,0,100007,0.069311,0.069311,0.078174,0.078174,0.074837,0.074837


In [43]:
app_test_bureau_group.head()

,SK_ID_CURR,bureau_CREDIT_ACTIVE_count,bureau_CREDIT_ACTIVE_count_norm,bureau_CREDIT_CURRENCY_count,bureau_CREDIT_CURRENCY_count_norm,bureau_CREDIT_TYPE_count,bureau_CREDIT_TYPE_count_norm
0,100001,0.555898,0.079414,0.547218,0.078174,0.523856,0.074837
1,100005,0.255080,0.085027,0.234522,0.078174,0.237682,0.079227
2,100013,0.277244,0.069311,0.312696,0.078174,0.261135,0.065284
3,100028,0.949601,0.079133,0.938087,0.078174,0.963899,0.080325
4,100038,0.000000,NaN,0.000000,NaN,0.000000,NaN


ดู correlation

In [44]:
target_corrs(app_train_bureau_group)

TARGET
SK_ID_CURR
bureau_CREDIT_ACTIVE_count
bureau_CREDIT_ACTIVE_count_norm
bureau_CREDIT_CURRENCY_count
bureau_CREDIT_CURRENCY_count_norm
bureau_CREDIT_TYPE_count
bureau_CREDIT_TYPE_count_norm


[('bureau_CREDIT_ACTIVE_count_norm', 0.07984838908758479),
 ('bureau_CREDIT_TYPE_count_norm', 0.06209404493245699),
 ('bureau_CREDIT_CURRENCY_count', -0.009972830198011267),
 ('bureau_CREDIT_CURRENCY_count_norm', 0.007476750384860392),
 ('bureau_CREDIT_ACTIVE_count', -0.003540131260544053),
 ('bureau_CREDIT_TYPE_count', -0.0024757998518446633),
 ('SK_ID_CURR', -0.0021084690181977176)]

In [45]:
app_train = app_train.merge(app_train_bureau_group.drop(columns = 'TARGET'), on = 'SK_ID_CURR', how='left')
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,bureau_AMT_CREDIT_SUM_sum,bureau_AMT_CREDIT_MAX_OVERDUE_min,bureau_AMT_CREDIT_MAX_OVERDUE_mean,bureau_AMT_CREDIT_MAX_OVERDUE_max,bureau_CREDIT_ACTIVE_count,bureau_CREDIT_ACTIVE_count_norm,bureau_CREDIT_CURRENCY_count,bureau_CREDIT_CURRENCY_count_norm,bureau_CREDIT_TYPE_count,bureau_CREDIT_TYPE_count_norm
0,100002,1,0.083459,0.101419,0.085002,0.079616,0,202500.0,406597.5,24700.5,...,865055.565,0.0,1681.029,5043.645,0.601636,0.075204,0.625392,0.078174,0.651381,0.081423
1,100003,0,0.083459,0.069993,0.085002,0.083249,0,270000.0,1293502.5,35698.5,...,1017400.500,0.0,0.000,0.000,0.300818,0.075204,0.312696,0.078174,0.325690,0.081423
2,100004,0,0.054783,0.101419,0.072437,0.079616,0,67500.0,135000.0,6750.0,...,189037.800,0.0,0.000,0.000,0.138622,0.069311,0.156348,0.078174,0.149673,0.074837
3,100006,0,0.083459,0.069993,0.085002,0.079616,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN
4,100007,0,0.083459,0.101419,0.085002,0.079616,0,121500.0,513000.0,21865.5,...,146250.000,0.0,0.000,0.000,0.069311,0.069311,0.078174,0.078174,0.074837,0.074837


In [46]:
app_test = app_test.merge(app_test_bureau_group, on = 'SK_ID_CURR', how='left')
app_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,bureau_AMT_CREDIT_SUM_sum,bureau_AMT_CREDIT_MAX_OVERDUE_min,bureau_AMT_CREDIT_MAX_OVERDUE_mean,bureau_AMT_CREDIT_MAX_OVERDUE_max,bureau_CREDIT_ACTIVE_count,bureau_CREDIT_ACTIVE_count_norm,bureau_CREDIT_CURRENCY_count,bureau_CREDIT_CURRENCY_count_norm,bureau_CREDIT_TYPE_count,bureau_CREDIT_TYPE_count_norm
0,100001,0.083459,0.069993,0.085002,0.079616,0,135000.0,568800.0,20560.5,450000.0,...,1453365.00,NaN,NaN,NaN,0.555898,0.079414,0.547218,0.078174,0.523856,0.074837
1,100005,0.083459,0.101419,0.085002,0.079616,0,99000.0,222768.0,17370.0,180000.0,...,657126.00,0.0,0.0,0.0,0.255080,0.085027,0.234522,0.078174,0.237682,0.079227
2,100013,0.083459,0.101419,0.072437,0.079616,0,202500.0,663264.0,69777.0,630000.0,...,2072280.06,19305.0,19305.0,19305.0,0.277244,0.069311,0.312696,0.078174,0.261135,0.065284
3,100028,0.083459,0.069993,0.085002,0.079616,2,315000.0,1575000.0,49018.5,1575000.0,...,1520875.08,0.0,0.0,0.0,0.949601,0.079133,0.938087,0.078174,0.963899,0.080325
4,100038,0.083459,0.101419,0.072437,0.083249,1,180000.0,625500.0,32067.0,625500.0,...,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN


# Applying operation to another frame
ลองไปดูที่ตาราง bureau_balance กันบ้าง

In [47]:
bureau_balance = pd.read_csv('../input/bureau_balance.csv')
bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [48]:
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,bureau_AMT_CREDIT_SUM_sum,bureau_AMT_CREDIT_MAX_OVERDUE_min,bureau_AMT_CREDIT_MAX_OVERDUE_mean,bureau_AMT_CREDIT_MAX_OVERDUE_max,bureau_CREDIT_ACTIVE_count,bureau_CREDIT_ACTIVE_count_norm,bureau_CREDIT_CURRENCY_count,bureau_CREDIT_CURRENCY_count_norm,bureau_CREDIT_TYPE_count,bureau_CREDIT_TYPE_count_norm
0,100002,1,0.083459,0.101419,0.085002,0.079616,0,202500.0,406597.5,24700.5,...,865055.565,0.0,1681.029,5043.645,0.601636,0.075204,0.625392,0.078174,0.651381,0.081423
1,100003,0,0.083459,0.069993,0.085002,0.083249,0,270000.0,1293502.5,35698.5,...,1017400.500,0.0,0.000,0.000,0.300818,0.075204,0.312696,0.078174,0.325690,0.081423
2,100004,0,0.054783,0.101419,0.072437,0.079616,0,67500.0,135000.0,6750.0,...,189037.800,0.0,0.000,0.000,0.138622,0.069311,0.156348,0.078174,0.149673,0.074837
3,100006,0,0.083459,0.069993,0.085002,0.079616,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN
4,100007,0,0.083459,0.101419,0.085002,0.079616,0,121500.0,513000.0,21865.5,...,146250.000,0.0,0.000,0.000,0.069311,0.069311,0.078174,0.078174,0.074837,0.074837


In [49]:
app_train_cols = ['SK_ID_CURR','TARGET']
app_train_temp = app_train[app_train_cols]

app_test_cols = ['SK_ID_CURR']
app_test_temp = app_test[app_test_cols]

bureau_cols = ['SK_ID_CURR', 'SK_ID_BUREAU']
bureau_index = bureau[bureau_cols]

skIdCurr = bureau_balance['SK_ID_BUREAU']
bureau_balance_obj_cols = bureau_balance.select_dtypes('object')
bureau_balance_obj_cols['SK_ID_BUREAU'] = skIdCurr

In [50]:
app_train_temp.head()

,SK_ID_CURR,TARGET
0,100002,1
1,100003,0
2,100004,0
3,100006,0
4,100007,0


In [51]:
bureau_balance_obj_cols.head()

,STATUS,SK_ID_BUREAU
0,C,5715448
1,C,5715448
2,C,5715448
3,C,5715448
4,C,5715448


In [52]:
bureau_balance['STATUS'].unique()

array(['C', '0', 'X', '1', '2', '3', '5', '4'], dtype=object)

In [53]:
bureau_index.head()

,SK_ID_CURR,SK_ID_BUREAU
0,215354,5714462
1,215354,5714463
2,215354,5714464
3,215354,5714465
4,215354,5714466


In [54]:
bureau_train = app_train_temp.merge(bureau_index, how='left')
bureau_test = app_test_temp.merge(bureau_index, how='left')

In [55]:
bureau_train.head()

,SK_ID_CURR,TARGET,SK_ID_BUREAU
0,100002,1,6158904.0
1,100002,1,6158905.0
2,100002,1,6158906.0
3,100002,1,6158907.0
4,100002,1,6158908.0


In [56]:
bureau_balance_train = bureau_train.merge(bureau_balance_obj_cols, how='left')
bureau_balance_test = bureau_test.merge(bureau_balance_obj_cols, how='left')

In [57]:
bureau_balance_train.head()

,SK_ID_CURR,TARGET,SK_ID_BUREAU,STATUS
0,100002,1,6158904.0,C
1,100002,1,6158904.0,C
2,100002,1,6158904.0,0
3,100002,1,6158904.0,0
4,100002,1,6158904.0,0


In [58]:
TargetEncoder(bureau_balance_train, bureau_balance_test)

In [59]:
bureau_balance_train.head()

,SK_ID_CURR,TARGET,SK_ID_BUREAU,STATUS
0,100002,1,6158904.0,0.063918
1,100002,1,6158904.0,0.063918
2,100002,1,6158904.0,0.075885
3,100002,1,6158904.0,0.075885
4,100002,1,6158904.0,0.075885


In [60]:
app_train_bureau_balance_group = bureau_balance_train.drop(columns=["TARGET",'SK_ID_CURR']).groupby('SK_ID_BUREAU').agg(['sum', 'mean'])
app_test_bureau_balance_group = bureau_balance_test.drop(columns=['SK_ID_CURR']).groupby('SK_ID_BUREAU').agg(['sum', 'mean'])

In [61]:
app_train_bureau_balance_group.head()

STATUS     
                sum mean
SK_ID_BUREAU            
5000000.0       0.0  NaN
5000001.0       0.0  NaN
5000002.0       0.0  NaN
5000003.0       0.0  NaN
5000004.0       0.0  NaN

In [62]:
RenameColumn(app_train_bureau_balance_group,'SK_ID_BUREAU', 'bureau_balance')
RenameColumn(app_test_bureau_balance_group,'SK_ID_BUREAU', 'bureau_balance')

In [63]:
# count non-nan entries
app_train_bureau_balance_group.count()

bureau_balance_STATUS_count         1465325
bureau_balance_STATUS_count_norm     523515
dtype: int64

In [64]:
app_train_bureau_balance_group2 = bureau_balance_train.merge(app_train_bureau_balance_group, left_on = 'SK_ID_BUREAU', right_index = True, how="left")
app_test_bureau_balance_group2 = bureau_balance_test.merge(app_test_bureau_balance_group, left_on ='SK_ID_BUREAU', right_index = True, how="left")

In [65]:
app_train_bureau_balance_group3 = app_train_bureau_balance_group2.drop(columns=["TARGET",'SK_ID_BUREAU']).groupby('SK_ID_CURR').agg(['sum', 'mean'])
app_test_bureau_balance_group3 = app_test_bureau_balance_group2.drop(columns=['SK_ID_BUREAU']).groupby('SK_ID_CURR').agg(['sum', 'mean'])

In [66]:
RenameColumn(app_train_bureau_balance_group3,'SK_ID_CURR', 'bureau_balance')
RenameColumn(app_test_bureau_balance_group3,'SK_ID_CURR', 'bureau_balance')

In [67]:
app_train_bureau_balance_group3.head()

,bureau_balance_STATUS_count,bureau_balance_STATUS_count_norm,bureau_balance_bureau_balance_STATUS_count_count,bureau_balance_bureau_balance_STATUS_count_count_norm,bureau_balance_bureau_balance_STATUS_count_norm_count,bureau_balance_bureau_balance_STATUS_count_norm_count_norm
SK_ID_CURR,,,,,,
100002,8.693234,0.079029,141.31595,1.28469,8.693234,0.079029
100003,0.000000,NaN,0.00000,0.00000,0.000000,NaN
100004,0.000000,NaN,0.00000,0.00000,0.000000,NaN
100006,0.000000,NaN,0.00000,NaN,0.000000,NaN
100007,0.000000,NaN,0.00000,0.00000,0.000000,NaN


#### function targetEncoder สำหรับตาราง bureau_balance โดยเฉพาะ จากการทดลองก่อนหน้านี้

In [68]:
def count_categoricalForBureauBalance(app_train, app_test, bureau, bureau_balance) :
    app_train_cols = ['SK_ID_CURR','TARGET']
    app_train_temp = app_train[app_train_cols]

    app_test_cols = ['SK_ID_CURR']
    app_test_temp = app_test[app_test_cols]

    bureau_cols = ['SK_ID_CURR', 'SK_ID_BUREAU']
    bureau_index = bureau[bureau_cols]

    SK_ID_BUREAU = bureau_balance['SK_ID_BUREAU']
    bureau_balance_obj_cols = bureau_balance.select_dtypes('object')
    bureau_balance_obj_cols['SK_ID_BUREAU'] = SK_ID_BUREAU
    
    bureau_train = app_train_temp.merge(bureau_index, how='left')
    bureau_test = app_test_temp.merge(bureau_index, how='left')
    
    TargetEncoder(bureau_balance_train, bureau_balance_test)
    
    app_train_bureau_balance_group = bureau_balance_train.drop(columns=["TARGET",'SK_ID_CURR']).groupby('SK_ID_BUREAU').agg(['sum', 'mean'])
    app_test_bureau_balance_group = bureau_balance_test.drop(columns=['SK_ID_CURR']).groupby('SK_ID_BUREAU').agg(['sum', 'mean'])
    
    RenameColumn(app_train_bureau_balance_group,'SK_ID_BUREAU', 'bureau_balance')
    RenameColumn(app_test_bureau_balance_group,'SK_ID_BUREAU', 'bureau_balance')
    
    app_train_bureau_balance_group = bureau_balance_train.merge(app_train_bureau_balance_group, left_on = 'SK_ID_CURR', right_index = True, how="left")
    app_test_bureau_balance_group = bureau_balance_test.merge(app_test_bureau_balance_group, left_on ='SK_ID_CURR', right_index = True, how="left")
    
    app_train_bureau_balance_group = app_train_bureau_balance_group.drop(columns=["TARGET",'SK_ID_BUREAU']).groupby('SK_ID_CURR').agg(['sum', 'mean'])
    app_test_bureau_balance_group = app_test_bureau_balance_group.drop(columns=['SK_ID_BUREAU']).groupby('SK_ID_CURR').agg(['sum', 'mean'])
    
    RenameColumn(app_train_bureau_balance_group,'SK_ID_CURR', 'bureau_balance')
    RenameColumn(app_test_bureau_balance_group,'SK_ID_CURR', 'bureau_balance')
    
    app_train_bureau_balance_group = app_train_temp.merge(app_train_bureau_balance_group, left_on = 'SK_ID_CURR', right_index = True, how="left")
    app_test_bureau_balance_group = app_test_temp.merge(app_test_bureau_balance_group, left_on ='SK_ID_CURR', right_index = True, how="left")
    
    import gc
    gc.enable()
    del app_train_temp, app_test_temp, app_train_cols, app_test_cols, bureau_index, bureau_train, bureau_test, SK_ID_BUREAU
    gc.collect()
    return app_train_bureau_balance_group, app_test_bureau_balance_group

In [69]:
bureau_balance_counts_train, bureau_balance_counts_test = count_categoricalForBureauBalance(app_train, app_test, bureau, bureau_balance)

สนใจเฉพาะ column bureau_balance_STATUS_count_norm

In [70]:
bureau_balance_counts_train.count()

SK_ID_CURR                                                    307511
TARGET                                                        307511
bureau_balance_STATUS_count                                   307511
bureau_balance_STATUS_count_norm                               92231
bureau_balance_bureau_balance_STATUS_count_count              307511
bureau_balance_bureau_balance_STATUS_count_count_norm              0
bureau_balance_bureau_balance_STATUS_count_norm_count         307511
bureau_balance_bureau_balance_STATUS_count_norm_count_norm         0
dtype: int64

#### จัดการข้อมูล column ที่เป็นตัวเลขของตาราง bureau_balance

In [71]:
bureau_balance_agg = agg_numeric(bureau_balance, group_var='SK_ID_BUREAU', df_name='bureau_balance')
bureau_balance_agg.head()

,bureau_balance_MONTHS_BALANCE_sum,bureau_balance_MONTHS_BALANCE_min,bureau_balance_MONTHS_BALANCE_mean,bureau_balance_MONTHS_BALANCE_max,bureau_balance_MONTHS_BALANCE_count,SK_ID_BUREAU
0,-4656,-96,-48.0,0,97,5001709
1,-3403,-82,-41.0,0,83,5001710
2,-6,-3,-1.5,0,4,5001711
3,-171,-18,-9.0,0,19,5001712
4,-231,-21,-10.5,0,22,5001713


In [72]:
bureau_by_loan = bureau_balance_agg.merge(bureau[['SK_ID_BUREAU','SK_ID_CURR']], on='SK_ID_BUREAU', how='left')
bureau_by_loan.head()

,bureau_balance_MONTHS_BALANCE_sum,bureau_balance_MONTHS_BALANCE_min,bureau_balance_MONTHS_BALANCE_mean,bureau_balance_MONTHS_BALANCE_max,bureau_balance_MONTHS_BALANCE_count,SK_ID_BUREAU,SK_ID_CURR
0,-4656,-96,-48.0,0,97,5001709,NaN
1,-3403,-82,-41.0,0,83,5001710,162368.0
2,-6,-3,-1.5,0,4,5001711,162368.0
3,-171,-18,-9.0,0,19,5001712,162368.0
4,-231,-21,-10.5,0,22,5001713,150635.0


In [73]:
bureau_balance_by_client = agg_numeric(bureau_by_loan.drop(columns=['SK_ID_BUREAU']), group_var='SK_ID_CURR', df_name='client')
bureau_balance_by_client.head()

,client_bureau_balance_MONTHS_BALANCE_sum_sum,client_bureau_balance_MONTHS_BALANCE_sum_min,client_bureau_balance_MONTHS_BALANCE_sum_mean,client_bureau_balance_MONTHS_BALANCE_min_sum,client_bureau_balance_MONTHS_BALANCE_mean_sum,client_bureau_balance_MONTHS_BALANCE_min_min,client_bureau_balance_MONTHS_BALANCE_mean_min,client_bureau_balance_MONTHS_BALANCE_min_mean,client_bureau_balance_MONTHS_BALANCE_mean_mean,client_bureau_balance_MONTHS_BALANCE_sum_max,...,client_bureau_balance_MONTHS_BALANCE_max_sum,client_bureau_balance_MONTHS_BALANCE_max_min,client_bureau_balance_MONTHS_BALANCE_max_mean,client_bureau_balance_MONTHS_BALANCE_max_max,client_bureau_balance_MONTHS_BALANCE_count_min,client_bureau_balance_MONTHS_BALANCE_sum_count,client_bureau_balance_MONTHS_BALANCE_count_mean,client_bureau_balance_MONTHS_BALANCE_count_max,client_bureau_balance_MONTHS_BALANCE_count_sum,SK_ID_CURR
0,-2800,-1326,-400.000000,-165,-82.5,-51,-25.5,-23.571429,-11.785714,-1,...,0,0,0.0,0,2,7,24.571429,52,172,100001.0
1,-2701,-632,-337.625000,-226,-175.0,-47,-39.5,-28.250000,-21.875000,-6,...,-124,-32,-15.5,0,4,8,13.750000,22,110,100002.0
2,-91,-78,-30.333333,-18,-9.0,-12,-6.0,-6.000000,-3.000000,-3,...,0,0,0.0,0,3,3,7.000000,13,21,100005.0
3,-3312,-2610,-1656.000000,-127,-92.0,-90,-72.5,-63.500000,-46.000000,-702,...,-57,-55,-28.5,-2,36,2,36.000000,36,72,100010.0
4,-6756,-2346,-1689.000000,-226,-113.0,-68,-34.0,-56.500000,-28.250000,-780,...,0,0,0.0,0,40,4,57.500000,69,230,100013.0


# ทำใหม่รวดเดียวทั้งหมด โดยรวม function ก่อนหน้านี้ที่ทำมา

In [74]:
def TargetEncoder(app_train,app_test) :
    for col in app_train :
        if app_train[col].dtype == 'object':
            cumsum = app_train.groupby(col)['TARGET'].sum()
            cumcnt = app_train.groupby(col)['TARGET'].count()
            for x in list(cumsum.index) :
                c = cumsum.loc[x]/cumcnt.loc[x]
                app_train.loc[app_train[col] == x, col] = c
                app_test.loc[app_test[col] == x, col] = c
            app_train[col] = app_train[col].astype('float64')
            app_test[col] = app_test[col].astype('float64')

In [75]:
def agg_numeric(df, group_var, df_name) :
    for col in df:
        if col != group_var and 'SK_ID' in col : # เป็น SK_ID แต่ไม่ใช่่สิ่งที่ต้องการจะ group ก็ให้ลบทิ้ง col นั้นไปเลย
            df = df.drop(columns = col)
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number') # ทำตารางใหม่ เอาข้อมูลที่เป็น ตัวเลข เท่านั้น
    numeric_df[group_var] = group_ids # ใส่สิ่งที่จะ group ไปด้วย
    
    agg = numeric_df.groupby(group_var).agg(['count','mean','max','min','sum']).reset_index()
    
    columns = [group_var] # เปลี่ยนชื่อตาราง
    for var in agg.columns.levels[0]:
        if var != group_var:
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))
    agg.columns = columns
    
    # Remove the columns with all redundant values
    _, idx = np.unique(agg, axis = 1, return_index=True)
    agg = agg.iloc[:,idx]
    
    return agg

In [76]:
def count_categoricalForBureau(app_train, app_test, bureau, group_var, df_name) :
    train_cols = ['TARGET'] + [group_var]
    app_train_temp = app_train.copy()
    app_train_temp = app_train_temp[train_cols]
    test_cols = [group_var]
    app_test_temp = app_test.copy()
    app_test_temp = app_test_temp[test_cols]
    
    skIdCurr = bureau[group_var]
    bureau_obj_cols = bureau.select_dtypes('object')
    bureau_obj_cols[group_var] = skIdCurr
    app_train_temp_bureau_obj_cols = app_train_temp.merge(bureau_obj_cols, how="left")
    app_test_temp_bureau_obj_cols = app_test_temp.merge(bureau_obj_cols, how="left")
    TargetEncoder(app_train_temp_bureau_obj_cols, app_test_temp_bureau_obj_cols)
    
    app_train_bureau_group = app_train_temp_bureau_obj_cols.drop(columns="TARGET").groupby(group_var).agg(['sum', 'mean'])
    app_test_bureau_group = app_test_temp_bureau_obj_cols.groupby(group_var).agg(['sum', 'mean'])
    
    RenameColumn(app_train_bureau_group,group_var, df_name)
    RenameColumn(app_test_bureau_group,group_var, df_name)
    
    app_train_bureau_group = app_train_temp.merge(app_train_bureau_group, left_on = group_var, right_index = True, how="left")
    app_test_bureau_group = app_test_temp.merge(app_test_bureau_group, left_on = group_var, right_index = True, how="left")
    
    return app_train_bureau_group, app_test_bureau_group

In [77]:
def RenameColumn(df, group_var, df_name):
    # Need to create new column names
    columns = []

    # Iterate through the variables names
    for var in df.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in ['count', 'count_norm']:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))

    #  Rename the columns
    df.columns = columns

In [78]:
def count_categoricalForBureauBalance(app_train, app_test, bureau, bureau_balance) :
    app_train_cols = ['SK_ID_CURR','TARGET']
    app_train_temp = app_train[app_train_cols]

    app_test_cols = ['SK_ID_CURR']
    app_test_temp = app_test[app_test_cols]

    bureau_cols = ['SK_ID_CURR', 'SK_ID_BUREAU']
    bureau_index = bureau[bureau_cols]

    SK_ID_BUREAU = bureau_balance['SK_ID_BUREAU']
    bureau_balance_obj_cols = bureau_balance.select_dtypes('object')
    bureau_balance_obj_cols['SK_ID_BUREAU'] = SK_ID_BUREAU
    
    bureau_train = app_train_temp.merge(bureau_index, how='left')
    bureau_test = app_test_temp.merge(bureau_index, how='left')
    
    TargetEncoder(bureau_balance_train, bureau_balance_test)
    
    app_train_bureau_balance_group = bureau_balance_train.drop(columns=["TARGET",'SK_ID_CURR']).groupby('SK_ID_BUREAU').agg(['sum', 'mean'])
    app_test_bureau_balance_group = bureau_balance_test.drop(columns=['SK_ID_CURR']).groupby('SK_ID_BUREAU').agg(['sum', 'mean'])
    
    RenameColumn(app_train_bureau_balance_group,'SK_ID_BUREAU', 'bureau_balance')
    RenameColumn(app_test_bureau_balance_group,'SK_ID_BUREAU', 'bureau_balance')
    
    app_train_bureau_balance_group = bureau_balance_train.merge(app_train_bureau_balance_group, left_on = 'SK_ID_CURR', right_index = True, how="left")
    app_test_bureau_balance_group = bureau_balance_test.merge(app_test_bureau_balance_group, left_on ='SK_ID_CURR', right_index = True, how="left")
    
    app_train_bureau_balance_group = app_train_bureau_balance_group.drop(columns=["TARGET",'SK_ID_BUREAU']).groupby('SK_ID_CURR').agg(['sum', 'mean'])
    app_test_bureau_balance_group = app_test_bureau_balance_group.drop(columns=['SK_ID_BUREAU']).groupby('SK_ID_CURR').agg(['sum', 'mean'])
    
    RenameColumn(app_train_bureau_balance_group,'SK_ID_CURR', 'bureau_balance')
    RenameColumn(app_test_bureau_balance_group,'SK_ID_CURR', 'bureau_balance')
    
    app_train_bureau_balance_group = app_train_temp.merge(app_train_bureau_balance_group, left_on = 'SK_ID_CURR', right_index = True, how="left")
    app_test_bureau_balance_group = app_test_temp.merge(app_test_bureau_balance_group, left_on ='SK_ID_CURR', right_index = True, how="left")
    
    import gc
    gc.enable()
    del app_train_temp, app_test_temp, app_train_cols, app_test_cols, bureau_index, bureau_train, bureau_test, SK_ID_BUREAU
    gc.collect()
    return app_train_bureau_balance_group, app_test_bureau_balance_group

In [79]:
import gc
gc.enable()
del app_train, app_test, bureau, bureau_balance, bureau_agg_new, app_train_bureau_group, app_test_bureau_group, bureau_balance_counts_train, bureau_balance_counts_test,bureau_balance_agg, bureau_by_loan, bureau_balance_by_client
gc.collect()

35

Load data

In [80]:
app_test = pd.read_csv('../input/application_test.csv')
app_train = pd.read_csv('../input/application_train.csv')
bureau = pd.read_csv('../input/bureau.csv')
bureau_balance = pd.read_csv('../input/bureau_balance.csv')

In [81]:
TargetEncoder(app_train,app_test)

นับข้อมูลตัวเลขในตาราง bureau

In [82]:
bureau_agg_new = agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), group_var='SK_ID_CURR', df_name='bureau')

นับข้อมูล categorical ในตาราง bureau

In [83]:
app_train_bureau_group, app_test_bureau_group = count_categoricalForBureau(app_train, app_test, bureau, group_var='SK_ID_CURR', df_name='bureau')

นับข้อมูล categorical ในตาราง bureau_balance

In [85]:
bureau_balance_counts_train, bureau_balance_counts_test = count_categoricalForBureauBalance(app_train, app_test, bureau, bureau_balance)

In [86]:
bureau_balance_counts_cols = ['SK_ID_CURR','bureau_balance_STATUS_count_norm']
bureau_balance_counts_train = bureau_balance_counts_train[bureau_balance_counts_cols]
bureau_balance_counts_test = bureau_balance_counts_test[bureau_balance_counts_cols]


นับข้อมูล ตัวเลข ในตาราง bureau_balance

In [87]:
bureau_balance_agg = agg_numeric(bureau_balance, group_var='SK_ID_BUREAU', df_name='bureau_balance')
#bureau_balance_agg.head()

นับ count mean max min sum ของการกู้ในแต่ละครั้ง (SK_ID_BUREAU)

In [88]:
bureau_by_loan = bureau_balance_agg.merge(bureau[['SK_ID_BUREAU','SK_ID_CURR']], on='SK_ID_BUREAU', how='left')
bureau_by_loan.head()

,bureau_balance_MONTHS_BALANCE_sum,bureau_balance_MONTHS_BALANCE_min,bureau_balance_MONTHS_BALANCE_mean,bureau_balance_MONTHS_BALANCE_max,bureau_balance_MONTHS_BALANCE_count,SK_ID_BUREAU,SK_ID_CURR
0,-4656,-96,-48.0,0,97,5001709,NaN
1,-3403,-82,-41.0,0,83,5001710,162368.0
2,-6,-3,-1.5,0,4,5001711,162368.0
3,-171,-18,-9.0,0,19,5001712,162368.0
4,-231,-21,-10.5,0,22,5001713,150635.0


นับ bureau_balance รายบุคคล

In [89]:
bureau_balance_by_client = agg_numeric(bureau_by_loan.drop(columns=['SK_ID_BUREAU']), group_var='SK_ID_CURR', df_name='client')
bureau_balance_by_client.head()

,client_bureau_balance_MONTHS_BALANCE_sum_sum,client_bureau_balance_MONTHS_BALANCE_sum_min,client_bureau_balance_MONTHS_BALANCE_sum_mean,client_bureau_balance_MONTHS_BALANCE_min_sum,client_bureau_balance_MONTHS_BALANCE_mean_sum,client_bureau_balance_MONTHS_BALANCE_min_min,client_bureau_balance_MONTHS_BALANCE_mean_min,client_bureau_balance_MONTHS_BALANCE_min_mean,client_bureau_balance_MONTHS_BALANCE_mean_mean,client_bureau_balance_MONTHS_BALANCE_sum_max,...,client_bureau_balance_MONTHS_BALANCE_max_sum,client_bureau_balance_MONTHS_BALANCE_max_min,client_bureau_balance_MONTHS_BALANCE_max_mean,client_bureau_balance_MONTHS_BALANCE_max_max,client_bureau_balance_MONTHS_BALANCE_count_min,client_bureau_balance_MONTHS_BALANCE_sum_count,client_bureau_balance_MONTHS_BALANCE_count_mean,client_bureau_balance_MONTHS_BALANCE_count_max,client_bureau_balance_MONTHS_BALANCE_count_sum,SK_ID_CURR
0,-2800,-1326,-400.000000,-165,-82.5,-51,-25.5,-23.571429,-11.785714,-1,...,0,0,0.0,0,2,7,24.571429,52,172,100001.0
1,-2701,-632,-337.625000,-226,-175.0,-47,-39.5,-28.250000,-21.875000,-6,...,-124,-32,-15.5,0,4,8,13.750000,22,110,100002.0
2,-91,-78,-30.333333,-18,-9.0,-12,-6.0,-6.000000,-3.000000,-3,...,0,0,0.0,0,3,3,7.000000,13,21,100005.0
3,-3312,-2610,-1656.000000,-127,-92.0,-90,-72.5,-63.500000,-46.000000,-702,...,-57,-55,-28.5,-2,36,2,36.000000,36,72,100010.0
4,-6756,-2346,-1689.000000,-226,-113.0,-68,-34.0,-56.500000,-28.250000,-780,...,0,0,0.0,0,40,4,57.500000,69,230,100013.0


In [90]:
previous_loan_counts = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU' : 'previous_loan_counts'})

เพิ่ม feature เข้าไป

In [91]:
original_features = list(app_train.columns)
print('Original Feature :', len(original_features))

Original Feature : 122


In [92]:
app_train = app_train.merge(bureau_agg_new, on = 'SK_ID_CURR', how='left')
app_test = app_test.merge(bureau_agg_new, on = 'SK_ID_CURR', how='left')
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,bureau_AMT_CREDIT_SUM_DEBT_mean,bureau_AMT_CREDIT_SUM_min,bureau_AMT_CREDIT_SUM_mean,bureau_AMT_CREDIT_SUM_DEBT_max,bureau_AMT_CREDIT_SUM_max,bureau_AMT_CREDIT_SUM_DEBT_sum,bureau_AMT_CREDIT_SUM_sum,bureau_AMT_CREDIT_MAX_OVERDUE_min,bureau_AMT_CREDIT_MAX_OVERDUE_mean,bureau_AMT_CREDIT_MAX_OVERDUE_max
0,100002,1,0.083459,0.101419,0.085002,0.079616,0,202500.0,406597.5,24700.5,...,49156.2,0.0,108131.945625,245781.0,450000.0,245781.0,865055.565,0.0,1681.029,5043.645
1,100003,0,0.083459,0.069993,0.085002,0.083249,0,270000.0,1293502.5,35698.5,...,0.0,22248.0,254350.125000,0.0,810000.0,0.0,1017400.500,0.0,0.000,0.000
2,100004,0,0.054783,0.101419,0.072437,0.079616,0,67500.0,135000.0,6750.0,...,0.0,94500.0,94518.900000,0.0,94537.8,0.0,189037.800,0.0,0.000,0.000
3,100006,0,0.083459,0.069993,0.085002,0.079616,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,0.083459,0.101419,0.085002,0.079616,0,121500.0,513000.0,21865.5,...,0.0,146250.0,146250.000000,0.0,146250.0,0.0,146250.000,0.0,0.000,0.000


In [93]:
app_train = app_train.merge(app_train_bureau_group.drop(columns = 'TARGET'), on = 'SK_ID_CURR', how='left')
app_test = app_test.merge(app_test_bureau_group, on = 'SK_ID_CURR', how='left')

In [94]:
app_train = app_train.merge(bureau_balance_by_client, on='SK_ID_CURR', how='left')
app_test = app_test.merge(bureau_balance_by_client, on='SK_ID_CURR', how='left')

In [95]:
app_train = app_train.merge(bureau_balance_counts_train, on='SK_ID_CURR', how='left')
app_test = app_test.merge(bureau_balance_counts_test, on='SK_ID_CURR', how='left')

In [96]:
app_train = app_train.merge(previous_loan_counts, on = 'SK_ID_CURR', how = 'left')
app_test = app_test.merge(previous_loan_counts, on = 'SK_ID_CURR', how = 'left')

# Fill the missing values with 0 
app_train['previous_loan_counts'] = app_train['previous_loan_counts'].fillna(0)
app_test['previous_loan_counts'] = app_test['previous_loan_counts'].fillna(0)

In [97]:
new_features = list(app_train.columns)
print('New Feature :', len(new_features))

New Feature : 207


In [98]:
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,client_bureau_balance_MONTHS_BALANCE_max_min,client_bureau_balance_MONTHS_BALANCE_max_mean,client_bureau_balance_MONTHS_BALANCE_max_max,client_bureau_balance_MONTHS_BALANCE_count_min,client_bureau_balance_MONTHS_BALANCE_sum_count,client_bureau_balance_MONTHS_BALANCE_count_mean,client_bureau_balance_MONTHS_BALANCE_count_max,client_bureau_balance_MONTHS_BALANCE_count_sum,bureau_balance_STATUS_count_norm,previous_loan_counts
0,100002,1,0.083459,0.101419,0.085002,0.079616,0,202500.0,406597.5,24700.5,...,-32.0,-15.5,0.0,4.0,8.0,13.75,22.0,110.0,0.079029,8.0
1,100003,0,0.083459,0.069993,0.085002,0.083249,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2,100004,0,0.054783,0.101419,0.072437,0.079616,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,100006,0,0.083459,0.069993,0.085002,0.079616,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,100007,0,0.083459,0.101419,0.085002,0.079616,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


## Missing Values

An important consideration is the missing values in the dataframe. Columns with too many missing values might have to be dropped. 

In [99]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df, print_info = False):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        if print_info:
            # Print some summary information
            print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
                "There are " + str(mis_val_table_ren_columns.shape[0]) +
                  " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [100]:
missing_train = missing_values_table(app_train)
missing_train.head(10)

,Missing Values,% of Total Values
bureau_AMT_ANNUITY_min,227502,74.0
bureau_AMT_ANNUITY_mean,227502,74.0
bureau_AMT_ANNUITY_max,227502,74.0
bureau_balance_STATUS_count_norm,215280,70.0
client_bureau_balance_MONTHS_BALANCE_mean_mean,215280,70.0
client_bureau_balance_MONTHS_BALANCE_min_sum,215280,70.0
client_bureau_balance_MONTHS_BALANCE_mean_sum,215280,70.0
client_bureau_balance_MONTHS_BALANCE_min_min,215280,70.0
client_bureau_balance_MONTHS_BALANCE_mean_min,215280,70.0
client_bureau_balance_MONTHS_BALANCE_min_mean,215280,70.0


We see there are a number of columns with a high percentage of missing values. There is no well-established threshold for removing missing values, and the best course of action depends on the problem. Here, to reduce the number of features, we will remove any columns in either the training or the testing data that have greater than 90% missing values.

In [101]:
missing_train_vars = list(missing_train.index[missing_train['% of Total Values'] > 90])
len(missing_train_vars)

0

Before we remove the missing values, we will find the missing value percentages in the testing data. We'll then remove any columns with greater than 90% missing values in either the training or testing data.
Let's now read in the testing data, perform the same operations, and look at the missing values in the testing data. We already have calculated all the counts and aggregation statistics, so we only need to merge the testing data with the appropriate data. 

In [102]:
missing_test = missing_values_table(app_test)
missing_test.head(10)

,Missing Values,% of Total Values
COMMONAREA_MEDI,33495,68.7
COMMONAREA_MODE,33495,68.7
COMMONAREA_AVG,33495,68.7
NONLIVINGAPARTMENTS_MEDI,33347,68.4
NONLIVINGAPARTMENTS_MODE,33347,68.4
NONLIVINGAPARTMENTS_AVG,33347,68.4
FONDKAPREMONT_MODE,32797,67.3
LIVINGAPARTMENTS_MEDI,32780,67.2
LIVINGAPARTMENTS_MODE,32780,67.2
LIVINGAPARTMENTS_AVG,32780,67.2


In [103]:
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,client_bureau_balance_MONTHS_BALANCE_max_min,client_bureau_balance_MONTHS_BALANCE_max_mean,client_bureau_balance_MONTHS_BALANCE_max_max,client_bureau_balance_MONTHS_BALANCE_count_min,client_bureau_balance_MONTHS_BALANCE_sum_count,client_bureau_balance_MONTHS_BALANCE_count_mean,client_bureau_balance_MONTHS_BALANCE_count_max,client_bureau_balance_MONTHS_BALANCE_count_sum,bureau_balance_STATUS_count_norm,previous_loan_counts
0,100002,1,0.083459,0.101419,0.085002,0.079616,0,202500.0,406597.5,24700.5,...,-32.0,-15.5,0.0,4.0,8.0,13.75,22.0,110.0,0.079029,8.0
1,100003,0,0.083459,0.069993,0.085002,0.083249,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2,100004,0,0.054783,0.101419,0.072437,0.079616,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,100006,0,0.083459,0.069993,0.085002,0.079616,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,100007,0,0.083459,0.101419,0.085002,0.079616,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [104]:
app_train.dtypes.value_counts()

float64    166
int64       41
dtype: int64

In [105]:
app_train.to_csv('train_part1-1.csv', index = False)
app_test.to_csv('test_part1-1.csv', index = False)